# 6 - Advanced topics: Understanding trackerdict structure

Tutorial 3 gives a good, detailed introduction to the trackerdict structure step by step.
Here is a condensed summary of functions you can use to explore the tracker dictionary.


### Steps:

<ol>
    <li> <a href='#step1'> Create a short Simulation + tracker dictionary beginning to end for 1 day </a></li>
    <li> <a href='#step2'> Explore the tracker dictionary </a></li>
    <li> <a href='#step3'> Explore Save Options </a></li>
</ol>

<a id='step 1'></a>

### 1. Create a short Simulation + tracker dictionary beginning to end for 1 day

In [1]:
import bifacial_radiance

simulationName = 'Tutorial 3'
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = False
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)

# Simulation range days
startdate = '11/06'     
enddate = '11/07'

# Cell Parameters
numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

demo = bifacial_radiance.RadianceObj(simulationName, path=testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readWeatherFile(epwfile)  
cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}
mymodule = demo.makeModule(name=moduletype, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                xgap=xgap, ygap=ygap, zgap=zgap, numpanels=numpanels, 
                cellLevelModuleParams=cellLevelModuleParams, 
                axisofrotationTorqueTube=axisofrotationTorqueTube)
sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}  
demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = mymodule['sceney'] / pitch, cumulativesky = cumulativesky)
demo.gendaylit1axis(startdate=startdate, enddate=enddate)
demo.makeScene1axis(moduletype=moduletype,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()
demo.analysis1axis()

path = C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!
Saving file EPWs\epw_temp.csv, # points: 8760

Module Name: Custom_Cell-Level_Module
REWRITING pre-existing module file. 
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Custom Cell-Level Module successfully created
Creating ~12 skyfiles.  Takes 1-2 minutes

C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:449: RuntimeWarning: invalid value encountered in arccos
  wc = np.degrees(np.arccos(temp))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:473: RuntimeWarning: invalid value encountered in arccos
  aoi = np.degrees(np.arccos(np.abs(np.sum(sun_vec*panel_norm, axis=0))))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:568: RuntimeWarning: invalid value encountered in remainder
  surface_azimuth = surface_azimuth % 360
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:572: RuntimeWarning: invalid value encountered in arccos
  surface_tilt = 90 - np.degrees(np.arccos(dotproduct))



Created 10 skyfiles in /skies/

Making ~10 .rad files for gendaylit 1-axis workflow (this takes a minute..)
10 Radfiles created in /objects/

Making 10 octfiles in root directory.
Created 1axis_11_06_08.oct
Created 1axis_11_06_09.oct
Created 1axis_11_06_10.oct
Created 1axis_11_06_11.oct
Created 1axis_11_06_12.oct
Created 1axis_11_06_13.oct
Created 1axis_11_06_14.oct
Created 1axis_11_06_15.oct
Created 1axis_11_06_16.oct
Created 1axis_11_06_17.oct
Linescan in process: 1axis_11_06_08_Front
Linescan in process: 1axis_11_06_08_Back
Saved: results\irr_1axis_11_06_08.csv
Index: 11_06_08. Wm2Front: 99.89242333333333. Wm2Back: 9.009616851851852
Linescan in process: 1axis_11_06_09_Front
Linescan in process: 1axis_11_06_09_Back
Saved: results\irr_1axis_11_06_09.csv
Index: 11_06_09. Wm2Front: 399.7332325925926. Wm2Back: 23.452134814814812
Linescan in process: 1axis_11_06_10_Front
Linescan in process: 1axis_11_06_10_Back
Saved: results\irr_1axis_11_06_10.csv
Index: 11_06_10. Wm2Front: 493.99228481

{'11_06_08': {'surf_azm': 90.0,
  'surf_tilt': 12.96,
  'theta': -12.96,
  'ghi': 76,
  'dhi': 70,
  'skyfile': 'skies\\sky2_37.5_-77.33_11_06_08.rad',
  'clearance_height': 1.8196120300464254,
  'radfile': 'objects\\1axis11_06_08_1.819_10_12.96_20x7_origin0,0.rad',
  'scene': <bifacial_radiance.main.SceneObj at 0x25ac2e68d68>,
  'octfile': '1axis_11_06_08.oct',
  'AnalysisObj': <bifacial_radiance.main.AnalysisObj at 0x25ac4038fd0>,
  'Wm2Front': [112.3098,
   18.742393333333332,
   113.9627,
   114.01853333333334,
   76.63301666666666,
   115.91313333333333,
   116.09366666666666,
   115.64063333333333,
   115.71793333333333],
  'Wm2Back': [9.150004666666668,
   8.867592,
   8.458989,
   8.124022666666667,
   11.90211,
   8.26887,
   8.569317666666667,
   8.764618,
   8.981027666666668],
  'backRatio': [0.0814703899061058,
   0.4731049411543765,
   0.07422529279059911,
   0.07125114819507535,
   0.15531105529507022,
   0.07133616723183597,
   0.07381318981061433,
   0.0757911986138786

<a id='step2'></a>

### 2. Explore the tracker dictionary

You can use any of the below options to explore the tracking dictionary. Copy it into an empty cell to see their contents.

In [3]:
demo.__dict__   # Shows all keys 

trackerkeys = sorted(demo.trackerdict.keys()) # get the trackerdict keys to see a specific hour.

demo.trackerdict[trackerkeys[0]] # This prints all trackerdict content
demo.trackerdict[trackerkeys[0]]['scene']  # This just prints that scene is a Scene object
demo.trackerdict[trackerkeys[0]]['scene'].__dict__ # This shows the Scene Object contents
demo.trackerdict[trackerkeys[0]]['scene'].scenex  # Addressing one of the variables in the Scene object
demo.trackerdict[trackerkeys[0]]['scene'].sceneDict # Printing the scene dictionary saved in the Scene Object
demo.trackerdict[trackerkeys[0]]['scene'].sceneDict['tilt'] # Addressing one of the variables in the scene dictionary
demo.trackerdict[trackerkeys[0]]['scene'].scene.__dict__ # Swhoing the scene dictionary inside the Scene Object values 

# Looking at the AnalysisObj results indivudally
demo.trackerdict[trackerkeys[0]]['AnalysisObj']  # This just prints that AnalysisObj is an Analysis object
demo.trackerdict[trackerkeys[0]]['AnalysisObj'].__dict__ # This shows the Analysis Object contents
demo.trackerdict[trackerkeys[0]]['AnalysisObj'].mattype # Addressing one of the variables in the Analysis Object

# Looking at the Analysis results Accumulated for the day:
demo.Wm2Back  # this value is the addition of every individual irradiance result for each hour simulated.

#  THREE WAYS OF CALLING THE SAME THING:
# (this might be cleaned up/streamlined in following releases.
demo.trackerdict[trackerkeys[0]]['scene'].scenex
demo.trackerdict[trackerkeys[0]]['scene'].moduleDict['scenex']
demo.trackerdict[trackerkeys[0]]['scene'].scene.scenex

1.046

<a id='step3'></a>

### 3. Explore Save Options

The following lines offer ways to save your trackerdict or your demo object.

In [4]:
demo.exportTrackerDict(trackerdict = demo.trackerdict, savefile = 'results\\test_reindexTrue.csv', reindex = False)
demo.save(savefile = 'results\\demopickle.pickle')


Saved to file results\demopickle.pickle
